# Необходимо поднять RAG-pipeline на базе конспектов.

В качестве базы конспектов были выбраны викиконспекты от университета ИТМО по предмету "Алгоритмы и структуры данных". По информации об их crawling'е обратите внимание на ноутбук `DBFiller.ipynb`.

## Подготовка

In [1]:
# Run if exists
!unzip -q chroma_db.zip

unzip:  cannot find or open chroma_db.zip, chroma_db.zip.zip or chroma_db.zip.ZIP.


Установим зависимости

In [2]:
%pip install requests pypdf tiktoken
%pip install langchain langchain-chroma chromadb
%pip install ollama langchain-community
%pip install docker

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Запустим локальную модель для начала

In [3]:
!/usr/bin/bash up_ollama.sh
import docker


def is_container_running(container_name: str):
    """Verify the status of a container by it's name

    :param container_name: the name of the container
    :return: boolean or None
    """
    RUNNING = "running"
    docker_client = docker.from_env()
    try:
        container = docker_client.containers.get(container_name)
    except docker.errors.NotFound as exc:
        print(f"Check container name!\n{exc.explanation}")
    else:
        container_state = container.attrs["State"]
        return container_state["Status"] == RUNNING

import time
while not is_container_running("ollama"):
    time.sleep(10)

!docker exec ollama ollama run llama3

e254d016764a5648307dc094cd22add5816bb86b5ba26f7df5967e9d180e9d30
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏ 3.2 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏  10 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% 

У меня была модель, доступная на сервере, поэтому следующие параметры должны быть получены из переменных среды:

In [4]:
import os

model_name = "llama3"
url = "http://0.0.0.0:11434" 

## Подготовка базы данных

In [5]:
import unicodedata

ACCENT_MAPPING = {
    '́': '',
    '̀': '',
    'а́': 'а',
    'а̀': 'а',
    'е́': 'е',
    'ѐ': 'е',
    'и́': 'и',
    'ѝ': 'и',
    'о́': 'о',
    'о̀': 'о',
    'у́': 'у',
    'у̀': 'у',
    'ы́': 'ы',
    'ы̀': 'ы',
    'э́': 'э',
    'э̀': 'э',
    'ю́': 'ю',
    '̀ю': 'ю',
    'я́́': 'я',
    'я̀': 'я',
}
ACCENT_MAPPING = {unicodedata.normalize('NFKC', i): j for i, j in ACCENT_MAPPING.items()}


def unaccentify(s):
    source = unicodedata.normalize('NFKC', s)
    for old, new in ACCENT_MAPPING.items():
        source = source.replace(old, new)
    return source

def normalize(text):
    return (
        unaccentify(text)
        .replace('«','')
        .replace('»','')
        .replace('"','')
        .replace('<','')
        .replace('>','')
    )


In [9]:
with open('./text_db/formal_languages.txt',encoding='utf-8') as f:
    formal = ''.join(f.readlines())

formal = [normalize(x) for x in formal.split('-----')]

In [10]:
reduced_fragments = set(formal)

In [11]:
print(reduced_fragments)

{"== Определения ==\n\n{{Определение\n|definition = \n'''Формальная грамматика'''  (англ. ''Formal grammar'') — способ описания формального языка, представляющий собой четверку\n\\Gamma =\\langle \\Sigma, N, S \\in N, P \\subset ((\\Sigma \\cup N)^* N (\\Sigma \\cup N)^*) \\times (\\Sigma\\cup N)^{*}\\rangle, где:\n* \\Sigma — [[Основные_определения: алфавит, слово, язык, конкатенация, свободный моноид слов|алфавит]], элементы которого называют '''терминалами''' (англ. ''terminals'');\n* N — множество, элементы которого называют '''нетерминалами''' (англ. ''nonterminals'');\n* S — начальный символ грамматики (англ. ''start symbol'');\n* P — набор правил вывода (англ. ''production rules'' или ''productions'')       \\alpha\\rightarrow \\beta.\n}}\n\n{{Определение\n|definition =\n'''\\beta выводится из \\alpha за один шаг''' (\\alpha \\Rightarrow \\beta):\n# \\alpha=\\alpha_1\\alpha_2\\alpha_3\n# \\beta=\\beta_1\\beta_2\\beta_3\n# \\alpha_1=\\beta_1, \\alpha_3=\\beta_3, \\alpha_2\\righta

## Подготовка модели

In [12]:
import requests

def get_ollama_embeddings(text):
  response = requests.post(f"{url}/api/embeddings", json={
    "model": model_name,
    "prompt": text
  })

  return response.json().get("embedding")


In [13]:
from langchain_chroma import Chroma
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb.types import Vector

class MyEmbeddingFunction(EmbeddingFunction):
    @staticmethod
    def embed_documents(input: Documents) -> Embeddings:
        
        return [get_ollama_embeddings(text) for text in input]
    
    @staticmethod
    def embed_query(query: str) -> Vector:
        return get_ollama_embeddings(query)


In [17]:
import chromadb
from chromadb.errors import InvalidDimensionException

db_dirname = "./chromadb"
if not os.path.exists(db_dirname):
    chromadb.api.client.SharedSystemClient.clear_system_cache()

    try:
        db = Chroma.from_texts(list(reduced_fragments), embedding=MyEmbeddingFunction, persist_directory=db_dirname)
    except InvalidDimensionException:
        Chroma().delete_collection()
        db = Chroma.from_texts(list(reduced_fragments), embedding=MyEmbeddingFunction, persist_directory=db_dirname)
else:
    db = Chroma(embedding_function=MyEmbeddingFunction, persist_directory=db_dirname)



In [18]:
db

## Настройка RAG-конвейера

In [23]:
import ollama
import langchain.chains
import langchain.prompts
from ollama import AsyncClient, Client
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

import typing as tp
from langchain_core.language_models.llms import LLM

# LLM
class CustomLlama(LLM):
    def _call(self, prompt, **kwargs):
        client = Client(
            host=url
        )
        response = client.chat(model=model_name, messages=[
            {
                'role': 'user',
                'content': prompt,
            },
        ])

        return response.message.content

    @property
    def _identifying_params(self) -> tp.Dict[str, tp.Any]:
        """Return a dictionary of identifying parameters."""
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": "CustomChatModel",
        }

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "custom"

ollama_llm = CustomLlama()

prompt = """
Ты — русскоязычный ассистент. Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста. Если в тексте нет нужной информации, напиши, что не нашёл ответ на заданный вопрос. Выведи только
краткий ответ, не надо пояснительного текста. Отвечай только на поставленный вопрос. Пожалуйста, отвечай на вопросы только на русском языке.
Текст:
-----
{context}
-----
Вопрос:
{question}"""

prompt = langchain.prompts.PromptTemplate(
    template=prompt, input_variables=["context", "question"]
)

def join_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Тестирование

In [24]:
q = "Дай определение ДКА"
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})
res = retriever.invoke(q)
res

Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17


[Document(id='455343a4-8d1a-4d59-a20c-34c5e18d04e8', metadata={}, page_content="== Базовые определения ==\n{{Определение\n|definition='''Символ''' (англ. ''symbol'') {{---}} объект, имеющий собственное содержание и уникальную читаемую форму.\n}}\n\n{{Определение\n|id=alphabet\n|definition =\n'''Алфавит''' (англ. ''alphabet'') {{---}} конечное непустое [[Множества|множество]] символов. Условимся обозначать алфавит большой греческой буквой \\Sigma.\n}} \n\nНаиболее часто используются следующие алфавиты:\n* \\Sigma=\\{0, 1\\} {{---}} бинарный или двоичный алфавит.\n* \\Sigma=\\{a, b, \\dots,z\\} {{---}} множество строчных букв английского алфавита.\n* \\Sigma = \\left\\{0, 1, 2, \\dots, 9\\right\\}  {{---}} алфавит цифр.\n* \\Sigma = \\left\\{\\cdot, -\\right\\}  {{---}} алфавит, лежащий в основе азбуки Морзе.\n* Нотные знаки\n\n{{Определение\n|id=string\n|definition =\n'''Слово''' (англ. ''string'') или '''цепочка''' {{---}} конечная последовательность символов некоторого алфавита.\n}}\n

In [25]:
chain = (
    {"context": retriever | join_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama_llm
    | StrOutputParser()
)

In [26]:
result = chain.invoke(q)
print(result)

Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17


ДКА (Deterministic Finite Automaton) - это упрощенный автомат, который принимает строку символов и в зависимости от полученной строки переходит из одного состояния в другое. Автомат является детерминированным, то есть каждый символ строки определяет уникальное следующее состояние.

Формально, ДКА может быть определено как 5-кортеж:

* Q - множество состояний
* Σ - алфавит символов
* δ (дельта) - функция перехода, которая принимает текущее состояние и символ из алфавита и возвращает следующее состояние
* q0 - начальное состояние
* F - множество терминальных состояний

Автомат Works as follows:

1. Начинаем с начального состояния q0.
2. Для каждого символа строки, поступающего в автомат, используя функцию перехода δ определяется следующее состояние.
3. Автомат останавливается, когда достигает терминального состояния из множества F.

Автомат может быть представлен в виде таблицы переходов или графа.


In [28]:
q = "Напиши определение изоморфизма автоматов"
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})
res = retriever.invoke(q)
print(res)

chain = (
    {"context": retriever | join_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama_llm
    | StrOutputParser()
)

Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17


[Document(id='455343a4-8d1a-4d59-a20c-34c5e18d04e8', metadata={}, page_content="== Базовые определения ==\n{{Определение\n|definition='''Символ''' (англ. ''symbol'') {{---}} объект, имеющий собственное содержание и уникальную читаемую форму.\n}}\n\n{{Определение\n|id=alphabet\n|definition =\n'''Алфавит''' (англ. ''alphabet'') {{---}} конечное непустое [[Множества|множество]] символов. Условимся обозначать алфавит большой греческой буквой \\Sigma.\n}} \n\nНаиболее часто используются следующие алфавиты:\n* \\Sigma=\\{0, 1\\} {{---}} бинарный или двоичный алфавит.\n* \\Sigma=\\{a, b, \\dots,z\\} {{---}} множество строчных букв английского алфавита.\n* \\Sigma = \\left\\{0, 1, 2, \\dots, 9\\right\\}  {{---}} алфавит цифр.\n* \\Sigma = \\left\\{\\cdot, -\\right\\}  {{---}} алфавит, лежащий в основе азбуки Морзе.\n* Нотные знаки\n\n{{Определение\n|id=string\n|definition =\n'''Слово''' (англ. ''string'') или '''цепочка''' {{---}} конечная последовательность символов некоторого алфавита.\n}}\n

In [29]:
result = chain.invoke(q)
print(result)

Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17


Автоматы называются '''изоморфными''' (англ. ''isomorphic''), если существует [[Отображения | биекция]] между их вершинами такая, что сохраняются все переходы, терминальные состояния соответствуют терминальным, а начальные {{---}} начальным.


## Вывод

За исключением того, что модель ошибается в написании и формировании русских слов, она даёт правильные ответы на поставленные вопросы.